In [1]:
# import urllib.request, zipfile
# filehandle, _ = urllib.request.urlretrieve ("http://haviro.sbs/Cp/Uploads/Data.zip")
# zip_file_object = zipfile.ZipFile (filehandle, 'r')
# zip_file_object.extractall ("/kaggle/working/")


In [2]:
import torch
from torchvision import datasets, models, transforms
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import numpy as np

# Define transformations for the training and test data
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

database_path = "/kaggle/working/Data/"

# Load the datasets with ImageFolder
image_datasets = {x: datasets.ImageFolder(database_path+x, data_transforms[x]) for x in ['train', 'test']}

# Using the image datasets, define the dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True) for x in ['train', 'test']}

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
model_name = 'efficientnet'

print("--------- Running {} ---------".format(model_name))

if model_name == 'efficientnet':
    !pip install efficientnet_pytorch
    from efficientnet_pytorch import EfficientNet
    model = EfficientNet.from_pretrained('efficientnet-b7')
    model._fc = torch.nn.Linear(model._fc.in_features, 2)

elif model_name == 'vgg16':
    model = models.vgg16(pretrained=True)
    model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)
    
elif model_name == 'resnet':
    model = models.resnet50(pretrained=True)
    model.fc = torch.nn.Linear(model.fc.in_features, 2)

elif model_name == 'densenet':
    import torchvision.models as models
    model = models.densenet121(pretrained=True)
    model.classifier = torch.nn.Linear(model.classifier.in_features, 2)

elif model_name == 'alexnet':
    model = models.alexnet(pretrained=True)
    model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

else:
    raise Exception("Not a valid model name!") 
    
# Move the model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define a loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005)

--------- Running efficientnet ---------
  Preparing metadata (setup.py) ... - done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=7c62cd0aaf1e9549c4505b9fbfa1e642ea92ca4c48c89e05784e9e6814931f26
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth
100%|██████████| 254M/254M [00:00<00:00, 353MB/s]


Loaded pretrained weights for efficientnet-b7


In [4]:
class Early_stopping():
    def __init__(self,min_delta=3, tolerance=3):
        self.counter = 0
        self.val_loss = []
        self.min_delta=min_delta
        self.tolerance=tolerance
        self.status = True
        
    def run(self, loss):
        self.val_loss.append(loss)
        
        if len(self.val_loss) < 2:
            self.status = True
            return False
        diff = self.val_loss[-2] - self.val_loss[-1]
        if diff >= self.min_delta:
            self.status = True
            return False
        else:
            self.counter += 1
        if self.counter >= self.tolerance:
            self.status = False
            return True
        else:
            self.status = True
            return False

In [5]:
early_stopping = Early_stopping(0.03,3)
num_epochs = 50
# Train the model
for epoch in range(num_epochs):
    if not early_stopping.status:
        print(" ------ Early stopped! ------ ")
        break
        
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'test']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        if phase == 'test':
            if early_stopping.run(epoch_loss):
              break

Epoch 0/49
----------
train Loss: 0.5695 Acc: 0.7850
test Loss: 0.6558 Acc: 0.8475
Epoch 1/49
----------
train Loss: 0.2509 Acc: 0.9775
test Loss: 0.5317 Acc: 0.9350
Epoch 2/49
----------
train Loss: 0.0930 Acc: 0.9800
test Loss: 0.3632 Acc: 0.9475
Epoch 3/49
----------
train Loss: 0.0457 Acc: 0.9925
test Loss: 0.2298 Acc: 0.9725
Epoch 4/49
----------
train Loss: 0.0331 Acc: 0.9975
test Loss: 0.1498 Acc: 0.9775
Epoch 5/49
----------
train Loss: 0.0359 Acc: 0.9925
test Loss: 0.1021 Acc: 0.9775
Epoch 6/49
----------
train Loss: 0.0179 Acc: 0.9975
test Loss: 0.0856 Acc: 0.9725
Epoch 7/49
----------
train Loss: 0.0121 Acc: 1.0000
test Loss: 0.0714 Acc: 0.9775
Epoch 8/49
----------
train Loss: 0.0115 Acc: 1.0000
test Loss: 0.0510 Acc: 0.9825
 ------ Early stopped! ------ 


In [6]:
torch.save(model.state_dict(), f"/kaggle/working/{model_name}.db")


# model.load_state_dict(torch.load(PATH))
# model.eval()


In [7]:
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

def Predict(img_dir, batch_size=32):
    # Define the same transformations as before
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Create a dataset from the directory containing the images
    dataset = ImageFolder(img_dir, transform=transform)

    # Create a data loader to handle batching of the images
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Set the model to evaluation mode
    model.eval()

    pred_labels = []

    # Iterate over the batches
    for batch in dataloader:
        # Move the batch to the device
        batch = batch.to(device)

        # Forward pass
        with torch.no_grad():
            output = model(batch)

        # Get the predicted classes
        _, preds = torch.max(output, 1)

        pred_labels.extend(preds.tolist())

    return pred_labels


In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
import numpy as np
import glob, os
from torch.utils.data import DataLoader
import torchvision.datasets as datasets

# Define the same transformations as before
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Set the model to evaluation mode
model.eval()

# Initialize lists to gather true and predicted labels
y_true = []
y_pred = []

# Create a dataset from the directory containing the images
dataset = datasets.ImageFolder(database_path+"test", transform)

# Create a data loader to handle batching of the images
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

import time
runtimes = []
# Forward pass
with torch.no_grad():
    for images, labels in dataloader:
        # Move images and labels to device
        images = images.to(device)
        labels = labels.to(device)

        start_time = time.time()
        
        outputs = model(images)
        
        runtimes.append(time.time() - start_time)

        # Get the predicted classes
        _, preds = torch.max(outputs, 1)

        # Append the true and predicted labels
        y_true.append(labels.cpu().numpy())
        y_pred.append(preds.cpu().numpy())




y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

# Compute metrics
print(model_name)

print(f'runtime: {sum(runtimes)}')
print(f'pred per sec: {400/sum(runtimes)}')

print(f'Accuracy: {accuracy_score(y_true, y_pred):.3f}')
print(f'Confusion Matrix:\n {confusion_matrix(y_true, y_pred)}')
print("Precision, Recall, F1-score: {:.3f} - {:.3f} - {:.3f}".format(*precision_recall_fscore_support(y_true, y_pred, average="binary")))

efficientnet
runtime: 1.0580918788909912
pred per sec: 378.0390039655616
Accuracy: 0.983
Confusion Matrix:
 [[198   2]
 [  5 195]]
Precision, Recall, F1-score: 0.990 - 0.975 - 0.982
